In [26]:
import pandas as pd
import numpy as np
import os
import re

In [27]:
def extract_lhg(filename: str):

    pattern = r"L(\d+)H(\d+)G(\d+)"
    match = re.search(pattern, filename)

    if not match:
        return None
    
    L, H, G = map(int, match.groups())
    return {"L": L, "H": H, "G": G}

In [28]:
def despike_signal(signal: np.ndarray):

    signal = np.asarray(signal, dtype=float)
    
    clean_signal = signal.copy()
    spike_mask = np.zeros_like(signal, dtype=bool)
    
    return clean_signal, spike_mask

In [ ]:
VELOCITY_COLUMNS = [
    "V1/X/E(cm/s)",
    "V2/Y/N(cm/s)",
    "V3/Z/U(cm/s)",
]

def process_csv_file(filename: str, output_suffix: str = "_clean"):

    print(f"Procesando archivo: {filename}")
    
    data_info = extract_lhg(filename)
    print("Info LHG:", data_info)

    file_path = './raw_data/' + filename
    
    df = pd.read_csv(file_path, sep="\t")
    
    for col in VELOCITY_COLUMNS:

        if col not in df.columns:
            print(f"Algo mal. columna {col} no encontrada en {filename}, se omite.")
            continue
        
        print(f"  → Despiking en columna: {col}")
        signal = df[col].values
        
        clean_signal, spike_mask = despike_signal(signal)
        
        clean_col = col + "_clean"
        mask_col = col + "_is_spike"
        
        df[clean_col] = clean_signal
        df[mask_col] = spike_mask
    
    base, ext = os.path.splitext(filename)
    output_name = f"./clean_data/{base}{output_suffix}{ext}"
    
    df.to_csv(output_name, index=False, sep="\t")
    print(f"  ✅ Guardado como: {output_name}\n")


In [30]:
csv_files = [file for file in os.listdir(os.getcwd() + '/raw_data') if file.endswith(".csv")]

print(f"Se encontraron {len(csv_files)} archivos CSV.\n")

for i, filename in enumerate(csv_files, start=1):
    print(f"[{i}/{len(csv_files)}]")
    process_csv_file(filename)

Se encontraron 30 archivos CSV.

[1/30]
Procesando archivo: L220H12G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 12, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H12G16_Probe_1_(A837F)._clean.csv

[2/30]
Procesando archivo: L220H15G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 15, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H15G16_Probe_1_(A837F)._clean.csv

[3/30]
Procesando archivo: L220H18G14_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 18, 'G': 14}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H18G14_Probe_1_(A837F)._clean.csv

[4/30]
Procesando archivo: L220H22G14_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 22, 'G': 14}
  → Despiki